In [2]:
! pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 390.2 kB/s eta 0:00:00a 0:00:01


### Exploring Pinecone
- Pinecone is a vector database that allows you to store, retrieve, and search high-dimensional vectors.

We will explore the Pinecone API and see how we can push our data to their servers.

##### GOAL: GraphRAG + Pinecone
Use GraphRAG techniques along with Pinecone's APIs. Hopefully that works!